### Data Scraping

In [3]:
import os
import re
import pickle
import time
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from urllib.request  import urlopen
from bs4             import BeautifulSoup

In [4]:
cwd = os.getcwd().replace('/notebooks','')
data_dir = os.path.join(cwd, 'data')

### Scraping Historical Game Data

In [5]:
#2017-18
#Done: [20171017-20180412]

#2016-17
#Done: [20161025-20170412]

#2015-16
#Done: [20151025-20150413]

#2014-15
#[20151018-20150425]

#2013-14
#[20131029-20140416]

In [6]:
start_date = '20131029'
end_date = '20140416'
season = '2013-14'
date_list = [d.strftime('%Y%m%d') for d in pd.date_range(start_date,end_date,index=False)]

In [9]:
url_parent = "https://www.basketball-reference.com"
url_boxscore = "https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}"

for date in tqdm(date_list):
    
    url_summaries = url_boxscore.format(month=date[4:6],day=date[6:8],year=date[0:4])
    soup_summaries = BeautifulSoup(urlopen(url_summaries),'html5lib')
    games = soup_summaries.find_all('div',class_='game_summary expanded nohover')
    
    print(date)
    
    for game in games:
        summary = {}
        
        host = game.find_all('table')[1].find_all('a')[1]['href'][7:10]
        
        winner = game.find('tr',class_='winner').find_all('td')
        loser = game.find('tr',class_='loser').find_all('td')
        
        summary['winner'] = [winner[0].find('a')['href'][7:10],int(winner[1].get_text())]
        summary['loser'] = [loser[0].find('a')['href'][7:10],int(loser[1].get_text())]
        

        url_game = url_parent+game.find('a',text='Box Score')['href']
        soup_game = BeautifulSoup(urlopen(url_game),'html5lib')
        
        
        tables = soup_game.find_all('table',limit=4)[2:]
        
        columns_basic = [th.get_text() for th in tables[0].find('thead').find_all('tr')[1].find_all('th')][1:]
        columns_advanced = [th.get_text() for th in tables[1].find('thead').find_all('tr')[1].find_all('th')][2:]
        
        game_columns = ['Name','Date','Team','Home','W','W_PTS','L','L_PTS']
        column_headers = game_columns + columns_basic + columns_advanced
        
        teams = ['winner','loser']
        basic_stat_template = 'box_{team}_basic'
        advanced_stat_template = 'box_{team}_advanced'

        for team in teams:
            
            if summary[team][0] == host:
                home = 1
            else:
                home = 0
            
            basic_stat = basic_stat_template.format(team=summary[team][0].lower())
            advanced_stat = advanced_stat_template.format(team=summary[team][0].lower())
            
            game_data = [date, summary[team][0], home,summary['winner'][0],
                         summary['winner'][1], summary['loser'][0],summary['loser'][1]]
            
            data_basic = soup_game.find('table',id=basic_stat).find('tbody').find_all('tr',class_=None)
            data_advanced = soup_game.find('table',id=advanced_stat).find('tbody').find_all('tr',class_=None)
            
            n = len(data_basic)
            
            player_names = [data_basic[i].find('a').get_text() for i in range(n)]
            
            player_data = []
            injury_keywords = ['Did Not Dress', 'Not With Team']
    
            for i in range(n):
                if data_basic[i].find('td').get_text() not in injury_keywords:
                    data = [player_names[i]] + game_data + \
                           [td.get_text() for td in data_basic[i].find_all('td')] + \
                           [td.get_text() for td in data_advanced[i].find_all('td')[1:]]
                            
                    player_data.append(data)
            
            df = pd.DataFrame(player_data,columns=column_headers)
            df.columns = df.columns.str.replace('%','_perc').str.replace('/','')
            df = df.fillna(0)
            df.loc[:,'FG':'+-'] = df.loc[:,'FG':'+-'].apply(pd.to_numeric)
            df['MP'] = [0.00 if ':' not in t else round(int(t.split(':')[0])+int(t.split(':')[1])/60, 2) for t in df['MP']] 
            df.to_csv(os.path.join(*[data_dir, 'Games', season, date+'-'+summary[team][0]+'.csv']), index=False)
            
        time.sleep(1)


HBox(children=(IntProgress(value=0, max=170), HTML(value='')))

20131029
20131030
20131031
20131101
20131102
20131103
20131104
20131105
20131106
20131107
20131108
20131109
20131110
20131111
20131112
20131113
20131114
20131115
20131116
20131117
20131118
20131119
20131120
20131121
20131122
20131123
20131124
20131125
20131126
20131127
20131128
20131129
20131130
20131201
20131202
20131203
20131204
20131205
20131206
20131207
20131208
20131209
20131210
20131211
20131212
20131213
20131214
20131215
20131216
20131217
20131218
20131219
20131220
20131221
20131222
20131223
20131224
20131225
20131226
20131227
20131228
20131229
20131230
20131231
20140101
20140102
20140103
20140104
20140105
20140106
20140107
20140108
20140109
20140110
20140111
20140112
20140113
20140114
20140115
20140116
20140117
20140118
20140119
20140120
20140121
20140122
20140123
20140124
20140125
20140126
20140127
20140128
20140129
20140130
20140131
20140201
20140202
20140203
20140204
20140205
20140206
20140207
20140208
20140209
20140210
20140211
20140212
20140213
20140214
20140215
20140216
2

### Scraping Historical Salary Data from RotoGuru

In [ ]:
start_date = '20141028'
end_date = '20150415'
season = '2014-15'
date_list = [d.strftime('%Y%m%d') for d in pd.date_range(start_date,end_date,index=False)]

In [10]:
url_roto = "http://rotoguru1.com/cgi-bin/hyday.pl?mon={month}&day={day}&year={year}&game=dk"

In [11]:
for date in tqdm(date_list):
    teams, positions, players, starters, salaries = [], [], [], [], []
    
    url_date = url_roto.format(month=date[4:6],day=date[6:8],year=date[0:4])
    soup = BeautifulSoup(urlopen(url_date),'html5lib')
    
    #Check if there were any games on a given date
    soup_table = soup.find('body').find('table', border="0", cellspacing="5")
    
    if soup_table.find('tbody') != None:
        soup_rows = soup_table.find('tbody').find_all('tr')
        
        for row in soup_rows:
            if row.find('td').has_attr('colspan') == False:
                if row.find('a').get_text() != '':
                    
                    position = row.find_all('td')[0].get_text()
                    
                    player_tmp = row.find('a').get_text().split(", ")
                    player = player_tmp[1] + ' ' + player_tmp[0]
                    
                    starter_tmp = row.find_all('td')[1].get_text()
                    if '^' in starter_tmp:
                        starter = 1
                    else:
                        starter =0
                        
                    salary_tmp = row.find_all('td')[3].get_text()
                    salary = re.sub('[$,]', '', salary_tmp)
                    
                    team = row.find_all('td')[4].get_text()

                    positions.append(position)
                    players.append(player)
                    starters.append(starter)
                    salaries.append(salary)
                    teams.append(team)
                
        df = pd.DataFrame({'Date': [date for i in range(len(players))], 
                           'Team': [team.upper() for team in teams],
                           'Starter': starters,
                           'Pos': positions,
                           'Name': players,
                           'Salary': salaries})
        
        df = df.loc[:,['Date','Team','Pos','Name','Starter','Salary']]
        
        df.to_csv(os.path.join(data_dir, 'DKSalary', season, 'salary_'+date+'.csv'), index=False)
        
    time.sleep(1)
    

HBox(children=(IntProgress(value=0, max=170), HTML(value='')))

AttributeError: 'NoneType' object has no attribute 'find'